## Call clusters
So I actually am not using parsimony clusters to call clusters. Insteed, I should be using some other type of clusters. Here I'm going to try to do that.


In [53]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from Bio import Phylo
from augur.utils import json_to_tree

In [86]:
#tree_path = '../../nextstrain_build/results/WA_20k/tree_raw.nwk'
tree_path = 'test3.nwk'
with open(tree_path,'r') as f:
    tree = Phylo.read(f, "newick")
    
with open('WA_20k_ko.tsv','r') as f:
    meta = pd.read_csv(f,sep='\t')
    
#path to tree json
#tree_file = '../measles/auspice/measles.json'

#with open(tree_file, 'r') as f:
    #tree_json = json.load(f)

#put tree in Bio.phylo format
#treeJ = json_to_tree(tree_json)

/var/folders/b5/2grxct1x69395r8j6vkk07bc0000gp/T/ipykernel_74774/608852175.py:7: DtypeWarning: Columns (50,60) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(f,sep='\t')


In [87]:
genes = ['ORF1a','ORF1b','S','ORF3a','E','M','ORF6','ORF7a','ORF7b','ORF8','N','ORF9b']

In [88]:
leafs = tree.get_terminals()

In [89]:
def replaceNaNs(df,genes):
    for gene in genes:
        df[gene+'_koType'] = np.where(df[gene+'_koType'].isnull(),'None',df[gene+'_koType'])
        df[gene+'_misStops'] = np.where(df[gene+'_misStops'].isnull(),'None',df[gene+'_misStops'])
        df[gene+'_deletions'] = df[gene+'_deletions'].str.replace(",\s+(?=[^\(]*\))",':', regex=True)
        df[gene+'_deletions'] = df[gene+'_deletions'].str.replace('[\[\]()]', '', regex=True)
    return df

def tabulate_names(tree):
    names = {}
    for idx, clade in enumerate(tree.find_clades()):
        if not clade.name:
            clade.name = str(idx)
        names[clade.name] = clade
    return names

def compareDels(deleted):
    linear = sorted(list(deleted))
    rolled = np.roll(linear,1)
    diffed = np.where((linear-rolled)>1)[0]
    if len(diffed):
        starts = []
        ends = []
        sizes = []
        nstart = linear[0]
        for value in diffed:
            nend = linear[value-1]
            sizes.append(nend-nstart+1)
            starts.append(nstart)
            ends.append(nend)
            nstart = linear[value]
        j = np.argmax(sizes)
        start = starts[j]
        end = ends[j]
    elif len(deleted):
        start = linear[0]
        end = linear[-1]
    else:
        start = 0
        end = 0 
    length = int(end-start + 1)
    return length,start,end

def extractDels(meta,clade,gene):
    locs = meta.loc[meta.strain==clade.name,gene+'_deletions']
    dels = [i for i in locs if i]
    missing = set()
    if len(dels):
        dels = dels[0].split(', ')
        for loc in dels:
            split = loc.split(':')
            asInt = [int(v) for v in split]
            if asInt[1] - asInt[0] >= 30:
                missing.update(range(asInt[0],asInt[1]+1))
    else:
        missing = {-1}
    return missing

def getNodeDels(subtree,meta,gene,states,leafs):
    for clade in subtree.find_clades():
        if clade in leafs:
            missing = extractDels(meta,clade,gene)
            states[clade.name] = missing
            return states[clade.name]
        else:
            flag = 0
            cluster = 0
            last = []
            current = set(range(0,29903+1))
            for child in clade:
                childState = getNodeDels(child,meta,gene,states,leafs)
                if len(childState) <= 1:
                    flag = 1
                else:
                    deleted = current.intersection(childState)
                    length,start,end = compareDels(deleted)
                    if length >= 30:
                        current = deleted
                        last.append(current)
                        cluster = 1
                    else:
                        flag = 1
                        if len(last):
                            current = last[-1]
                        else:
                            current = set(range(0,29903+1))
            if flag == 0:
                states[clade.name] = current
                return current
            elif cluster == 1:
                current.add(-1)
                states[clade.name] = current
            else:
                states[clade.name] = {-1}
            return {-1}
        
def getClustersDels(states,subtree):
    clusters = {}
    count = 1
    for clade in subtree.find_clades():
        if clade not in leafs:
            state = states[clade.name]
            if len(state) > 1:
                compared = state - {-1}
                clusters[count] = {}
                clusters[count]['strains'] = []
                for child in clade.find_clades():
                    if child in clade:
                        childState = states[child.name]
                        if -1 not in childState:
                            deleted = compared.intersection(childState)
                            length,start,end = compareDels(deleted)
                            if length >= 30:
                                compared = deleted
                                terms = child.get_terminals()
                                named = [leaf.name for leaf in terms]
                                clusters[count]['strains'].extend(named)
                                clusters[count]['deletion'] = str(start)+':'+str(end)
                depths = [len(clade.trace(clade,child)) for child in clusters[count]['strains']]
                clusters[count]['maxDepth'] = max(depths)
                count += 1
    return clusters
            
def getNodeStops(subtree,meta,gene,states,leafs):
    for clade in subtree.find_clades():
        if clade in leafs:
            if meta.loc[meta.strain==clade.name][gene+'_koType'].values[0] == 'earlyStop':
                states[clade.name] = [meta.loc[meta.strain==clade.name,gene+'_misStops'].values[0]]
            else:
                states[clade.name] = ['None']
            return states[clade.name]
        else:
            childStates = set()
            for child in clade:
                childStates.update(set(getNodeStops(child,meta,gene,states,leafs)))
            if len(childStates)==1:
                states[clade.name] = list(childStates)
                return list(childStates)
            else:
                states[clade.name] = list(childStates)
                return ['None']
            
def getClustersStops(states,subtree):
    clusters = {}
    count = 1
    for clade in subtree.find_clades():
        if clade not in leafs:
            state = states[clade.name]
            if len(state) > 1:
                for i in state:
                    if i != 'None':
                        clusters[count] = {}
                        clusters[count]['strains'] = []
                        for child in clade.find_clades():
                            if child in clade:
                                childState = states[child.name]
                                if len(childState) == 1:
                                    if i == childState[0]:
                                        terms = child.get_terminals()
                                        named = [leaf.name for leaf in terms]
                                        clusters[count]['strains'].extend(named)
                                        clusters[count]['mut']=i
                        count += 1
    return clusters


def getNodeFinch(subtree,meta,gene,states,leafs):
    for clade in subtree.find_clades():
        if clade in leafs:
            if meta.loc[meta.strain==clade.name][gene+'_koType'].values[0] == 'earlyStop':
                states[clade.name] = [meta.loc[meta.strain==clade.name,gene+'_misStops'].values[0]]
            else:
                states[clade.name] = ['None']
            return states[clade.name]
        else:
            childStates = []
            for child in clade:
                childStops  = set(getNodeFinch(child,meta,gene,states,leafs))
                childStates.append(childStops)
                
            intersect = set.intersection(*childStates)
            if not len(intersect):
                union = set.union(*childStates)
                nodeState = union
            else:
                nodeState = intersect  
            states[clade.name] = list(nodeState)
            return list(nodeState)

def getClusterLeafs(value,states,subtree,named):
    for child in subtree.find_clades():
        if child in subtree:
            childState = states[child.name]
            if value in childState:
                if child in leafs:
                    named.append(child.name)
                else:
                    getClusterLeafs(value,states,child,named)
    
            
def getClustersFinch(states,subtree):
    clusters = {}
    count = 1
    for clade in subtree.find_clades():
        if clade not in leafs:
            state = states[clade.name]
            for i in state:
                if i != 'None':
                    clusters[count] = {}
                    clusters[count]['strains'] = []
                    clusters[count]['mut'] = i
                    getClusterLeafs(i,states,clade,clusters[count]['strains'])
                    depths = [len(clade.trace(clade,child)) for child in clusters[count]['strains']]
                    clusters[count]['maxDepth'] = max(depths)
                    count += 1
    return clusters

def trimClusters(clusters):
    clustered = set()
    rid = []
    for k,v in clusters.items():
        values = set(v['strains'])
        if values.issubset(clustered):
            rid.append(k)
        else:
            clustered.update(values)
    for k in rid:
        clusters.pop(k)
    return clusters

def trimClustersMut(clusters):
    clustered = {}
    rid = []
    for k,v in clusters.items():
        mut = v['mut']
        values = set(v['strains'])
        if mut in clustered:
            if values.issubset(clustered[mut]):
                rid.append(k)
            else:
                clustered[mut].update(values)
        else:
            clustered[mut] = set()
    for k in rid:
        clusters.pop(k)
    return clusters

def saveClusters(stopC, delC,path,gene):
    clusts = []
    strains = []
    muts = []
    dels = []
    depths = []
    kotype = []
    for i,k in enumerate(stopC.keys()):
        samples = stopC[k]['strains']
        mut = stopC[k]['mut']
        depth = stopC[k]['maxDepth']
        length = len(samples)
        muts.extend([mut]*length)
        dels.extend(['None']*length)
        depths.extend([depth]*length)
        strains.extend(samples)
        clusts.extend([i+1]*length)
        kotype.extend(['earlyStop']*length)
    maxClust = i+1
    for j,l in enumerate(delC.keys()):
        samples = delC[l]['strains']
        delet = delC[l]['deletion']
        depth = delC[l]['maxDepth']
        length = len(samples)
        dels.extend([delet]*length)
        muts.extend(['None']*length)
        depths.extend([depth]*length)
        strains.extend(samples)
        clusts.extend([maxClust+j+1]*length)
        kotype.extend(['bigDeletion']*length)
    df = pd.DataFrame({'strain':strains,'cluster':clusts,'depth':depths,gene+'_misStops':muts,gene+'_deletions':dels,gene+'_koType':kotype})
    with open(path, 'w') as f:
        df.to_csv(f,sep='\t',index=False)
        
def get_parent(tree, child_clade):
    node_path = tree.get_path(child_clade)
    return node_path[-2]

meta = replaceNaNs(meta,genes)
names = tabulate_names(tree)

In [84]:
finch = {}
getNodeFinch(tree,meta,'ORF8',finch,leafs)
finchClusters = getClustersFinch(finch,tree)
finchTrimmed = trimClusters(finchClusters)

In [90]:
## NEED TO FIX PROBLEM WHERE WAY TOO MANY NODES ARE GETTING CALLED AS CLUSTERS B/C getting all children of node, not just the ones that only have deletions.

dels = {}
getNodeDels(tree,meta,'ORF8',dels,leafs)
delClusters = getClustersDels(dels,tree)
trimClusters(delClusters)
finch = {}
getNodeFinch(tree,meta,'ORF8',finch,leafs)
finchClusters = getClustersFinch(finch,tree)
trimClustersMut(finchClusters)
saveClusters(finchClusters,delClusters,'testClusters3.tsv','ORF8')

In [66]:
delClusters

{1: {'strains': ['USA/WA-UW-58958/2021',
   'USA/WA-S3677/2021',
   'USA/WA-S4673/2020',
   'USA/WA-S4158/2020',
   'USA/WA-UW-46944/2020',
   'USA/WA-CDC-4010417-001/2020',
   'USA/WA-S3152/2020',
   'USA/WA-S3984/2021',
   'USA/WA-S4063/2021',
   'USA/WA-S4102/2021',
   'USA/WA-S4112/2021',
   'USA/WA-S4097/2021',
   'USA/WA-S4283/2021',
   'USA/WA-S4455/2021',
   'USA/WA-MG_LHPOC80/2021'],
  'deletion': '27913:28061',
  'maxDepth': 5},
 2: {'strains': ['USA/WA-UW-29751/2020'],
  'deletion': '27822:28215',
  'maxDepth': 2},
 3: {'strains': ['USA/MT-UMGC-00001/2021'],
  'deletion': '27728:28233',
  'maxDepth': 2},
 9: {'strains': ['USA/DE-DHSS-B1069946/2021'],
  'deletion': '28070:28104',
  'maxDepth': 2},
 10: {'strains': ['USA/WA-CDC-LC0000715/2020'],
  'deletion': '27872:28699',
  'maxDepth': 2},
 11: {'strains': ['USA/WA-CDC-LC0004314/2021'],
  'deletion': '27872:28699',
  'maxDepth': 2},
 12: {'strains': ['USA/WA-S4165/2020'],
  'deletion': '27527:28088',
  'maxDepth': 2}}

In [38]:
finchTrimmed

{1: {'strains': ['Bangladesh/TND-04-0574/2021']},
 2: {'strains': ['Mozambique/CERI-KRISP-K030823/2021']},
 3: {'strains': ['Nigeria/LSB_COV2-07-21-0543/2021']},
 4: {'strains': ['Portugal/PT16357/2021']},
 5: {'strains': ['Iraq/Erbil-30/2021', 'Iraq/Erbil-7/2021']},
 7: {'strains': ['USA/WA-PHL-001173/2021']},
 8: {'strains': ['Bulgaria/21BG-NC_011549_R19/2021']},
 9: {'strains': ['USA/AL-CDC-QDX31823205/2021']},
 10: {'strains': ['DRC/INRB-RDC-647/2021']},
 11: {'strains': ['USA/WA-CDC-UW21102069596/2021']},
 12: {'strains': ['USA/WA-CDC-LC0385794/2021']},
 13: {'strains': ['USA/WA-CDC-UW21070721342/2021',
   'USA/WA-UW-21073050999/2021',
   'USA/WA-PHL-001436/2021',
   'USA/WA-CDC-FG-135969/2021',
   'USA/WA-PHL-004118/2021',
   'USA/WA-S9558/2021',
   'USA/NH-CDCBI-CRSP_2AWJIEAQ2R75DZP3/2021',
   'USA/WA-Altius-ALTCOV-ECRIKHX5IKMZMEGL/2021',
   'USA/WA-PHL-001516/2021',
   'USA/WA-S19290/2021',
   'USA/WA-CDC-FG-082235/2021',
   'USA/WA-CDC-QDX30811256/2021',
   'USA/WA-Altius-ALTC

In [52]:
delTrimmed

{1: {'strains': ['USA/WA-CDC-LC0003360/2021'], 'deletion': '27872:28699'},
 2: {'strains': ['USA/WA-S4115/2020'], 'deletion': '27925:27964'},
 3: {'strains': ['USA/WA-S5423/2021',
   'USA/WA-S5416/2021',
   'USA/WA-S5170/2021'],
  'deletion': '27928:28088'}}

In [81]:
stops = {}
getNodeStops(tree,meta,'ORF8',stops,leafs)

['None']

In [82]:
clusters = getClustersStops(stops,tree)

In [285]:
trimClusters(clusters)

{1: {'strains': ['USA/WA-UW-62666/2021'], 'mut': '27959-27961:CAG>TAG'}}

In [289]:
saveClusters(clusters,'testClusters.tsv','ORF8')

In [275]:
stops

{'USA/WA-UW-31592/2020': ['None'],
 'USA/WA-CDC-4010405-001/2020': ['None'],
 'USA/WA-UW-46934/2020': ['None'],
 'USA/WA-UW-68591/2021': ['None'],
 'USA/WA-UW-62826/2021': ['None'],
 '6': ['None'],
 '4': ['None'],
 '2': ['None'],
 'USA/OR-OSPHL00975/2020': ['None'],
 'USA/WA-OHSU-5660/2020': ['None'],
 'USA/WA-UW-59042/2021': ['None'],
 '11': ['None'],
 'USA/WA-S3758/2020': ['None'],
 'USA/WA-PHL-030472/2020': ['None'],
 'USA/WA-PHL-029809/2020': ['None'],
 'USA/WA-S3575/2020': ['None'],
 'USA/WA-S3624/2020': ['None'],
 '21': ['None'],
 '19': ['None'],
 'USA/WA-UW-65896/2021': ['None'],
 'USA/WA-UW-64070/2021': ['None'],
 'USA/WA-PHL-000060/2021': ['None'],
 '24': ['None'],
 '18': ['None'],
 '16': ['None'],
 '14': ['None'],
 '9': ['None'],
 'USA/WA-S3733/2020': ['None'],
 'USA/WA-S3727/2020': ['None'],
 '29': ['None'],
 'USA/WA-CDC-4010400-001/2020': ['None'],
 'USA/MT-Walk-1672/2020': ['None'],
 'USA/WA-PHL-028408/2020': ['None'],
 'USA/WA-S3120/2020': ['None'],
 'USA/WA-UW-61814/2021

In [25]:
for clade in tree.get_nonterminals():
    state = finch[clade.name]
    if state != ['None']:
        print(state)
        

['None', '28052-28054:TCA>TGA']
['27971-27973:CAA>TAA', 'None']
['None', '27947-27949:GAA>TAA']
['28208-28210:GAA>TAA', 'None']
['28208-28210:GAA>TAA', 'None']
['28208-28210:GAA>TAA']
['None', '27947-27949:GAA>TAA']
['27944-27946:CAA>TAA', 'None']
['28091-28093:TCT>TAA', 'None']
['27956-27958:TTA>TAA', 'None']
['28178-28180:GGT>TAG', '28142-28144:TTA>TAA', 'None']
['28208-28210:GAA>TAA', 'None']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA']
['28208-28210:GAA>TAA', 'None']
['27944-27946:CAA>TAA', 'None']
['28199-28201:TCG>TAG', 'None']
['27971-27973:CAA>TAA', 'None']
['None', '28220-28222:GAG>TAG']
['28067-28069:GAA>TAA', 'None']
['28106-28108:CAG>TAG', '

In [80]:
for clade in leafs:
    state = states[clade.name]
    if len(state) > 1:
        print(state)
    elif state != ['None']:
        print(state)

['BigDeletion']
['BigDeletion']
['BigDeletion']
['BigDeletion']
['earlyStop']
['BigDeletion']
